In [2]:
import requests
import re
import json
from tqdm import tqdm

# Metacritic API url
url = "https://mcgqlapi.com/api"

# Using readlines()
shows_file = open('tv-shows.txt', 'r')
shows_lines = shows_file.readlines()

shows = {}
elementos = []

for show_line in shows_lines:
    if (show_line[0] != '"'):
        try:
        
            info = show_line.split(", ")

            titulo, ano = info[0].split("(")
            network = info[1].strip()

            titulo = titulo.strip()
            ano = ano.strip(")")

            adaptation = "Original"
            if info[-1].strip().endswith("Adaptation)"):
                generos = [genero.strip() for genero in info[2:-3]]
                produtora = info[-3].strip()
                classificacao = info[-2].strip()
                if info[-1].strip().endswith("Book Adaptation)"):
                    adaptation = "Based on a book"
                if info[-1].strip().endswith("Comic Book Adaptation)"):
                    adaptation = "Based on a comic book"
                if info[-1].strip().endswith("Film Adaptation)"):
                    adaptation = "Based on a movie"
            else:
                generos = [genero.strip() for genero in info[2:-2]]
                produtora = info[-2].strip()
                classificacao = info[-1].strip()
                classificacao = classificacao.strip(")")
        except IndexError:
            # Skip malformed line
            continue
        nova_string = f'"{titulo}" ({ano}) - {network} - {produtora} - {", ".join(generos)} - {classificacao} - {adaptation}'
        
    else:
        info = show_line.split(" - ")
        titulo_ano = info[0]
        produtora = info[1]
        genero = info[2]
        classificacao = info[3]
        adaptation = info[4].strip()
        if adaptation == "Not a book adaptation":
            adaptation = "Original"
        titulo, ano = titulo_ano[1:-1].split(" (")
        network = "None"

        nova_string = f'"{titulo}" ({ano}) - {network} - {produtora} - {genero} - {classificacao} - {adaptation}'
    elementos.append(nova_string)

# Obtain information using API
for show_line in elementos:
    lista = show_line.split(" - ")
    titulo_ano = lista[0]
    titulo, ano = titulo_ano.split(" (")
    titulo = titulo.replace('"', '')
    ano = ano.strip(")").split("-")
    try:
        # criar um dicionário com as informações obtidas
        show = {
            "title": titulo,
            "year": "-".join(ano),
            "network": lista[1],
            "studio": lista[2],
            "genre": lista[3],
            "rating": lista[4],
            "adaptation": lista[5]
        }
        
        if titulo not in shows:
            # adicionar o dicionário ao dicionário principal
            shows[titulo] = {"data": {"tv-show": show}}
        else:
            print(titulo)
    except IndexError:
        # Skip malformed line
        continue

with open('tv-shows.json', 'w') as output_file:
    output_file.write(json.dumps(shows))

This Is Us
The Crown
The Boys
The Haunting of Hill House
The Marvelous Mrs. Maisel
The Sopranos
The Handmaid's Tale
The West Wing
The Big Bang Theory
House of Cards
Breaking Bad
Stranger Things
Friends
The Handmaid's Tale
Stranger Things
Master of None
Big Little Lies
The Crown
The Leftovers
The Expanse
The Man in the High Castle
The Killing
The Good Place
The OA
Love
Atlanta
The Leftovers
Mr. Robot
The Crown
Better Things
The Handmaid's Tale
American Gods
The Marvelous Mrs. Maisel
Mindhunter
Big Little Lies
Dear White People
Ozark
Patriot
The Umbrella Academy
Watchmen
Almost Family
The I-Land


In [3]:
import json
import pandas as pd
import csv


# Abra o arquivo JSON e carregue o conteúdo em uma variável
with open('tv-shows.json', 'r') as file:
    data = json.load(file)

with open('tv-shows.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Type', 'ReleaseYear', 'FinalYear', 'Network', 'Label/Studio', 'Genre', 'Rating', 'Adaptation'])

    for show_title, show_data in data.items():
        show = show_data['data']
        year = show['tv-show']['year'].split('-')
        final = 'None'
        if (len(year)>1):
            final = year[1]
        release = year[0]
        show = show_data['data']
        if((show.get('tv-show'))):
            writer.writerow([
                show['tv-show']['title'],
                "Tv-Show",
                release,
                final,
                show['tv-show']['network'],
                show['tv-show']['studio'],
                show['tv-show']['genre'],
                show['tv-show']['rating'],
                show['tv-show']['adaptation']
            ])

In [4]:
pd.read_csv("tv-shows.csv")

,Title,Type,ReleaseYear,FinalYear,Network,Label/Studio,Genre,Rating,Adaptation
0,Game of Thrones,Tv-Show,2011,2019,HBO,Warner Bros.,"Drama, Fantasy",TV-MA,Based on a book
1,Breaking Bad,Tv-Show,2008,2013,AMC,Sony Pictures Television,Drama,TV-MA,Original
2,Friends,Tv-Show,1994,2004,NBC,Warner Bros.,Comedy,TV-PG,Original
3,The Sopranos,Tv-Show,1999,2007,HBO,Warner Bros.,Drama,TV-MA,Original
4,Stranger Things,Tv-Show,2016,2022,Netflix,None,"Horror, Science Fiction",None,Based on a book
...,...,...,...,...,...,...,...,...,...
237,Indebted,Tv-Show,2020,None,None,Sony Pictures Television,Comedy,TV-14,Original
238,The Baker and the Beauty,Tv-Show,2020,None,None,Universal Television,"Comedy, Drama",TV-14,Based on a TV show
239,Council of Dads,Tv-Show,2020,None,None,Jerry Bruckheimer Television,Drama,TV-14,Based on a book
240,The Secret of Skinwalker Ranch,Tv-Show,2020,None,None,Painless Productions,Reality-TV,TV-14,Original
